 # Notebook for Downloading ds865.0 Data.

In [1]:
import sys, os
import requests

 ## First, we need to authenticate

Compatibility for python2 and getpass not installed

In [2]:
try:
    import getpass
    input = getpass.getpass
except:
    try:
        input = raw_input
    except:
        pass

Now, we need your password.

In [3]:
pswd = input('password: ')

password:  ··········


In [12]:
values = {'email' : 'cdoganay@purdue.edu', 'passwd' : pswd, 'action' : 'login'}
login_url = 'https://rda.ucar.edu/cgi-bin/login'

In [5]:
ret = requests.post(login_url, data=values)
if ret.status_code != 200:
    print('Bad Authentication')
    print(ret.text)
    exit(1)

In [13]:
dspath = 'https://rda.ucar.edu/data/ds865.0/'
filelist = ['tsur_ncar']

Change this if you want your files saved somewhere other than the current directory 

In [14]:
save_dir = ''

 ## Now to download the files

In [15]:
for file in filelist:
    filename = dspath + file
    outfile = save_dir + os.path.basename(filename)
    print('Downloading', file)
    req = requests.get(filename, cookies = ret.cookies, allow_redirects=True)
    open(outfile, 'wb').write(req.content)

### Once you have downloaded the data, the next part can help you plot it.

In order to plot this data, you may need to install librariesIn order to plot this data, you may need to install libraries. The easiest way to do this is to use conda, however any method of getting the following libraries will work.

In [16]:
import xarray as xr # used for reading the data.
import matplotlib.pyplot as plt # used to plot the data.
import ipywidgets as widgets # For ease in selecting variables.
import cartopy.crs as ccrs # Used to georeference data.

In [26]:
filelist_arr = [save_dir + os.path.basename(file) for file in filelist]
selected_file = widgets.Dropdown(options=filelist_arr, description='data file')
display(selected_file)

Dropdown(description='data file', options=('tsur_ncar',), value='tsur_ncar')

In [32]:
# Now to load in the data to xarray
ds = xr.open_dataset(selected_file.value,engine='scipy')

TypeError: Error: /Users/cansudoganay/Downloads/tsur_ncar is not a valid NetCDF 3 file
            If this is a NetCDF4 file, you may need to install the
            netcdf4 library, e.g.,

            $ pip install netcdf4
            

In [24]:
# Helper methods# Define function to get standard dimensions
def get_time(dataset):
    for _,cur_coord in dataset.coords.items:
        if cur_coord.attrs['standard_name'] == 'time':
            return cur_coord
def get_lat(dataset):
    for _,cur_coord in dataset.coords.items:
        if cur_coord.attrs['standard_name'] == 'longitude':
            return cur_coord
def get_lon(dataset):
    for _,cur_coord in dataset.coords.items:
        if cur_coord.attrs['standard_name'] == 'latitude':
            return cur_coord

def get_primary(dataset):
    primary_variables = {}
    coords = dataset.coords.keys()
    highest_dims = 0
    for cur_key,cur_var in dataset.variables.items():
        if cur_key not in coords:
            primary_variables[cur_key] = cur_var
    return primary_variables 

In [ ]:
var = widgets.Dropdown(
    options=get_primary(ds).keys(),
    description='Variable')
display(var)

In [ ]:
proj = ccrs.Mercator()
plt.gcf().set_size_inches(20,10)
ax = plt.axes(projection=proj)
data_slice = ds[var.value].isel(time=0)
data_slice.plot.contourf(ax=ax, transform=ccrs.PlateCarree())
ax.set_global()
ax.coastlines()